# Q5

In [23]:
import pandas as pd 
data = pd.read_excel("../../data/xyseries.xls")

(a)

In [24]:
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf



In [29]:
def analyze_acf(x:pd.Series, confidence_interval = 0.95, qstat = True): 
    values = acf (x, qstat = qstat, alpha = (1-confidence_interval))
    df = pd.DataFrame(values).T 
    df.columns = ['ACF','CI'+str(confidence_interval),'qstat','pvalue']
    
    return df 

In [30]:
d = analyze_acf(data.loc[:,"X"])
d

,ACF,CI0.95,qstat,pvalue
0,1.0,"[1.0, 1.0]",336.394459,0.0
1,0.817783,"[0.7301302758233357, 0.9054347839386521]",604.414793,0.0
2,0.729225,"[0.5952136556548029, 0.8632367939233936]",804.037089,0.0
3,0.628703,"[0.4670551415456956, 0.7903517985534811]",948.026537,0.0
4,0.53342,"[0.3539658291516083, 0.7128742069135237]",1063.316385,0.0
5,0.476827,"[0.2855790683436673, 0.6680758828897919]",1152.132785,0.0
6,0.418093,"[0.21791937114216134, 0.61826720911199]",1222.078372,0.0
7,0.370652,"[0.1638781340386509, 0.5774265668459512]",1284.917405,0.0
8,0.350962,"[0.13914493941740952, 0.5627796105747418]",1343.346204,0.0
9,0.338078,"[0.12183945075026573, 0.5543172560235401]",1401.003687,0.0


In [11]:
acf = acf(data.loc[:,"X"],qstat=True,alpha = 0.05 )


(array([1.        , 0.81778253, 0.72922522, 0.62870347, 0.53342002,
        0.47682748, 0.41809329, 0.37065235, 0.35096227, 0.33807835,
        0.33549729, 0.33473003, 0.32251952, 0.31717133, 0.30703032,
        0.30955917, 0.3177272 , 0.31135091, 0.30074866, 0.24861754,
        0.21281113, 0.21432929, 0.18244334, 0.17543259, 0.16204981,
        0.15238511, 0.13782752]),
 array([[ 1.        ,  1.        ],
        [ 0.73013028,  0.90543478],
        [ 0.59521366,  0.86323679],
        [ 0.46705514,  0.7903518 ],
        [ 0.35396583,  0.71287421],
        [ 0.28557907,  0.66807588],
        [ 0.21791937,  0.61826721],
        [ 0.16387813,  0.57742657],
        [ 0.13914494,  0.56277961],
        [ 0.12183945,  0.55431726],
        [ 0.11523488,  0.55575971],
        [ 0.11057588,  0.55888419],
        [ 0.09455738,  0.55048166],
        [ 0.08573003,  0.54861263],
        [ 0.07227333,  0.5417873 ],
        [ 0.07173709,  0.54738125],
        [ 0.07682929,  0.55862511],
        [ 0.06

In [25]:
acf_vals, q_stat, p_vals, confint = acf(data.loc[:, "X"], qstat=True, alpha=0.05)
print(p_vals)

[ 336.39445854  604.41479297  804.03708948  948.02653671 1063.31638482
 1152.13278525 1222.07837224 1284.91740504 1343.34620388 1401.00368693
 1458.51512398 1512.01661864 1563.86469088 1612.55022899 1662.1431096
 1714.49557578 1764.87194597 1811.97337391 1844.22801277 1867.91016437
 1891.98155973 1909.45996742 1925.65477907 1939.50203753 1951.77262033
 1961.83190918]


In [26]:
temp = acf(data.loc[:, "X"], qstat=True, alpha=0.05)
pd.DataFrame(temp)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,0.817783,0.729225,0.628703,0.53342,0.476827,0.418093,0.370652,0.350962,0.338078,...,0.311351,0.300749,0.248618,0.212811,0.214329,0.182443,0.175433,0.16205,0.152385,0.137828
1,"[1.0, 1.0]","[0.7301302758233357, 0.9054347839386521]","[0.5952136556548029, 0.8632367939233936]","[0.4670551415456956, 0.7903517985534811]","[0.3539658291516083, 0.7128742069135237]","[0.2855790683436673, 0.6680758828897919]","[0.21791937114216134, 0.61826720911199]","[0.1638781340386509, 0.5774265668459512]","[0.13914493941740952, 0.5627796105747418]","[0.12183945075026573, 0.5543172560235401]",...,"[0.06725463386171401, 0.5554471877232254]","[0.05362005153158836, 0.5478772603014797]","[-0.00130721240845999, 0.49854229745277684]","[-0.03900657437027086, 0.464628833012366]","[-0.03886638884961788, 0.46752497246334423]","[-0.07214242654843855, 0.43702911144229445]","[-0.08015569904754963, 0.4310208826151686]","[-0.09446195320749676, 0.41856156472539163]","[-0.1049119757285343, 0.4096822001686812]","[-0.12016202175649454, 0.39581706675660105]"
2,336.394459,604.414793,804.037089,948.026537,1063.316385,1152.132785,1222.078372,1284.917405,1343.346204,1401.003687,...,1811.973374,1844.228013,1867.910164,1891.98156,1909.459967,1925.654779,1939.502038,1951.77262,1961.831909,None
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None
